In [117]:
import pandas as pd
import matplotlib.pyplot as plt
from WindPy import w
w.start()
from function import *
from nav_research import NavResearch
from pathlib import Path
import os
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

In [118]:
data_path = r"data\旭诺CTA三号净值20250109.xlsx"
strategy = "量化CTA"
fund_name = "旭诺CTA三号"
benchmark_code = "NH0100.NHF"
benchmark_name = "南华商品指数"
threshold = -0.05

demo = NavResearch(data_path,strategy,fund_name,benchmark_code,benchmark_name,threshold)
df_nav, df_return, df_drawdown = demo.get_data()

In [119]:
data = df_nav[["date", "nav_adjusted"]]

In [129]:
import numpy as np
import pandas as pd
from typing import Literal

def calc_nav_rtn(nav: np.ndarray, types: Literal["log", "simple"] = "log"):
    if types == "simple":
        rtn = nav[1:] / nav[:-1] - 1
    elif types == "log":
        rtn = np.log(nav[1:] / nav[:-1])
    else:
        raise ValueError("types参数错误")
    return np.insert(rtn, 0, np.nan)
def win_ratio_stastics(nav: np.ndarray, date: np.ndarray[np.datetime64]):
    """
    目前只支持月度胜率统计
    """
    assert len(nav) == len(date), "nav和date长度不一致"
    
    nav_data = pd.DataFrame({"日期": date, "累计净值": nav})
    # 找到日期序列中每个月的最后一天
    nav_data["year_month"] = nav_data["日期"].dt.to_period("M")
    monthly_rtn = nav_data.drop_duplicates(
        subset="year_month", keep="last"
    ).reset_index(drop=True)

    monthly_rtn["rtn"] = calc_nav_rtn(monthly_rtn["累计净值"].values, types="simple")
    monthly_rtn = monthly_rtn.copy()
    monthly_rtn.iloc[0, monthly_rtn.columns.get_loc("rtn")] = (
        monthly_rtn.iloc[0]["累计净值"] - 1
    )
    monthly_rtn["year"] = monthly_rtn["日期"].dt.year
    monthly_rtn["month"] = monthly_rtn["日期"].dt.month
    monthly_rtn = monthly_rtn.pivot_table(
        index="year", columns="month", values="rtn", aggfunc="sum"
    )

    monthly_rtn.columns = [f"{x}月" for x in monthly_rtn.columns]
    monthly_rtn.index.name = None
    monthly_rtn["年度总收益"] = monthly_rtn.apply(lambda x: np.prod(x + 1) - 1, axis=1)
    monthly_rtn["月度胜率"] = monthly_rtn.apply(
        lambda x: (x >= 0).sum() / (~np.isnan(x)).sum(), axis=1
    )
    # 保留四位小数
    monthly_rtn = monthly_rtn.map(lambda x: round(x, 4))
    for col in monthly_rtn.columns:
        monthly_rtn[col] = monthly_rtn[col].map(lambda x: f"{x:.2%}")

    # 将NAN变成NULL
    if (monthly_rtn.iloc[0, :] == "nan%").sum() + 3 == monthly_rtn.shape[1] and monthly_rtn.iloc[0, monthly_rtn.shape[1] - 3] == "0.000%":
        monthly_rtn = monthly_rtn.iloc[1:]
    return monthly_rtn.replace("nan%", "")

In [130]:
win_ratio_stastics(nav=data["nav_adjusted"],date=data["date"])

,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,年度总收益,月度胜率
2019,,,,,,-0.03%,-0.04%,-5.30%,1.34%,2.82%,-3.64%,-0.91%,-5.85%,25.00%
2020,2.04%,15.60%,23.45%,-0.18%,8.08%,3.81%,14.95%,-1.56%,-7.21%,-0.70%,3.29%,3.33%,81.47%,69.23%
2021,5.06%,8.72%,-0.15%,-4.49%,-1.35%,-2.58%,3.30%,-5.70%,14.84%,6.28%,0.29%,-2.06%,22.29%,53.85%
2022,3.32%,-3.32%,-6.26%,0.01%,-0.65%,4.15%,5.99%,-6.11%,-3.01%,-6.58%,-0.03%,5.82%,-7.58%,38.46%
2023,6.50%,-2.00%,0.20%,9.19%,0.47%,-1.12%,5.34%,-0.67%,-1.22%,3.19%,-1.73%,-0.92%,17.79%,53.85%
2024,0.16%,-0.06%,12.07%,3.77%,-0.19%,-3.94%,-0.26%,-1.28%,31.39%,1.99%,10.37%,0.80%,63.81%,61.54%
2025,3.95%,,,,,,,,,,,,3.95%,100.00%


In [93]:
nav_month_end['monthly_return'] = (nav_month_end['nav_adjusted'] - nav_month_end['nav_adjusted'].shift(1)) / nav_month_end['nav_adjusted'].shift(1)
nav_month_end.dropna(inplace=True)

In [94]:
nav_month_end['year'] = nav_month_end['date'].dt.year
nav_month_end['month'] = nav_month_end['date'].dt.month

In [110]:
month_list = ['1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月',"胜率"]
year_list = nav_month_end["year"].unique()
month_return = pd.DataFrame(columns = month_list,index = year_list)
month_return

,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,胜率
2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
for _, row in nav_month_end.iterrows():
    year = row["date"].year
    month_str = f"{row["date"].month}月"
    month_return.loc[year, month_str] = row["monthly_return"]
    month_return.loc[year, "胜率"] = (
        nav_month_end.loc[
            (nav_month_end["year"] == year) & (nav_month_end["monthly_return"] > 0)
        ]["monthly_return"].count()
    ) / nav_month_end[nav_month_end["year"] == year]["monthly_return"].count()

In [112]:
for i in month_return.columns:
    month_return[i] = month_return[i].apply(lambda x: f"{x:.2%}")

In [113]:
month_return.index.name = "分月度业绩"
month_return = month_return.reset_index(drop=False)

In [114]:
month_return

,分月度业绩,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,胜率
0,2019,nan%,nan%,nan%,nan%,nan%,nan%,-0.04%,-5.30%,1.34%,2.82%,-3.64%,-0.91%,33.33%
1,2020,2.04%,15.60%,23.45%,-0.18%,8.08%,3.81%,14.95%,-1.56%,-7.21%,-0.70%,3.29%,3.33%,66.67%
2,2021,5.06%,8.72%,-0.15%,-4.49%,-1.35%,-2.58%,3.30%,-5.70%,14.84%,6.28%,0.29%,-2.06%,50.00%
3,2022,3.32%,-3.32%,-6.26%,0.01%,-0.65%,4.15%,5.99%,-6.11%,-3.01%,-6.58%,-0.03%,5.82%,41.67%
4,2023,6.50%,-2.00%,0.20%,9.19%,0.47%,-1.12%,5.34%,-0.67%,-1.22%,3.19%,-1.73%,-0.92%,50.00%
5,2024,0.16%,-0.06%,12.07%,3.77%,-0.19%,-3.94%,-0.26%,-1.28%,31.39%,1.99%,10.37%,0.80%,58.33%
6,2025,3.95%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,100.00%


In [25]:
def multi_fund_comparison(tables,fund_name):
    data1 = tables[0]
    data2 = tables[1].head(1)
    data3 = pd.DataFrame(columns=[2019,2020,2021,2022,2023,2024])
    data4 = tables[2]
    years = [2019, 2020, 2021, 2022, 2023, 2024]
    for year in years:
        data3[year] = data4.loc[data4["分年度业绩"] == year, f"{fund_name}_收益"].values
    data = pd.concat([data1,data2,data3], axis=1)
    data.drop(columns=["基准指数","整体业绩"], inplace=True)
    return data

In [31]:
basic_info = load_data("产品目录.xlsx")
data = pd.DataFrame()
files_list_series = pd.Series([i for i in Path("./data").rglob("*.xlsx")])
for row in basic_info.itertuples(index=False, name=None):
    nav_df_path = files_list_series[files_list_series.apply(lambda x: row[3] in x.stem)]
    assert len(nav_df_path) == 1, "找到多个文件"
    demo = NavResearch(nav_df_path.item(),row[0],row[3],row[4],row[5],row[6])
    demo.get_data()
    tables = demo.get_analysis_table()
    nav_df = multi_fund_comparison(tables,row[3])
    data = pd.concat([data,nav_df], axis=0)

草本优益1号无法推断频率,自动转为周度
济海实恒对冲二号无法推断频率,自动转为周度
均成CTA1号无法推断频率,自动转为周度
弘源多元化CTA无法推断频率,自动转为周度
宏锡量化CTA7号无法推断频率,自动转为周度
会世元丰CTA1号无法推断频率,自动转为周度
远澜翠柏1号无法推断频率,自动转为周度


In [33]:
data.to_excel('data.xlsx', index=False)